<a href="https://colab.research.google.com/github/UPstartDeveloper/DS-1.1-Data-Analysis/blob/master/Notebooks/Top_Spending_Users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [InterviewQs](https://www.interviewqs.com/): "Top spending users"

Suppose you have the following [dataset](https://docs.google.com/spreadsheets/d/1DrvkAWnO1psWkFN1YVt891sHe4yHl4ljNPUVlCsI95M/edit#gid=2039795889) which contains which contains:

**(1st tab)**: a list of items purchased by a given user, 

**(2nd tab)**: a mapping which maps the item_id to the item name and price

**(3rd tab)**: a matrix that formats data from sheet 1 into a matrix with users in rows and the number of each item_id purchased.

Given these 3 data sets, can you create a list of users who have spent the most?


**Clarifications**:

*Assume*
1. Return a list all user ids
2. Sorted in descendng order
3. Spent the most in DOLLARS, not quantitiy
4. Algorithm should be stable - 
  1. if two users spent ==, they go in the order they appear in sheet

5. Assume the user_ids are randomly generated
6. only care about the user_ids that show in sheet three

*Intuition:*
- 1st tab - wHAT items each user bought
- 2nd tab - what each items cost
- 3rd tab - the QUANTITIES of each sale

Approach:

1. map each user to the dollars they spent
  1. iterate through user_ids in SHEET 3
  2. Look up the items they bought [SHEET 1]
  3. for each of those items:
    1. look up the quantitiy that user the bought [3]
    2. multiply quantity * unit. price [2] 
  4. map the price in the dict, to a list of users, who have bought that much
2. sort by the keys in the dict, greatest to leas
3. merge all the users into another, larger list
4. return the sorted list

Edge Cases:
1. No users --> return []
2. no sales --> return all user ids [all bought zero] 
3. BIG dataset --> not worry for now, can think more about the datapipeline as needed


```
Small Input Test: First 3 rows of Sheet 3

dollar_users = {
  100: [100002]
}

100002 --> bought 42,45,22,33,15,11
total_spent = 0, 1
1 * 1 = 1
```

# Solution

## Importing Packages

In [1]:
import pandas as pd

## Getting the Datasets in Memory

My approach to this: 

1. Save the Google Spreadsheet above as an Excel spreadhsheet
2. Upload it someplace in Google Drive where I had permissions
3. Mount my Google Drive onto this Colab notebook
4. copy the path to the notebook (apparently it has to tbe the absolute URL, but I'm not sure) into the `URL` variable below

I am sure there are other ways to do this though!

In [2]:
URL = "/content/drive/MyDrive/Colab Notebooks/InterviewQs/grocery_shopping_data.xlsx"
items_sold, prices, quantities_sold = (
    pd.read_excel(URL, sheet_name=0),
    pd.read_excel(URL, sheet_name=1),
    pd.read_excel(URL, sheet_name=2)
)

### Test to Make Sure the Data Loaded in OK

In [3]:
items_sold.head()

,user_id,id
0,222087,"27,26"
1,1343649,"6,47,17"
2,404134,"18,12,23,22,27,43,38,20,35,1"
3,1110200,"9,23,2,20,26,47,37"
4,224107,"31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4..."


In [18]:
prices.head()

,item_name,Item_id,price
0,sugar,1,2
1,lettuce,2,1
2,pet items,3,2
3,baby items,4,4
4,waffles,5,2


In [5]:
quantities_sold.head()

,user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,100002,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,1000074,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,2,1,0,0,1,0,2,1,0,1,0,1,1,1,0,0,0,1,1,0,2,0,1,1,2,0,0,1,0,1,1,0,1
2,100009,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2,0,1,0,0,0,1,0,0,0
3,1000096,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0
4,1000103,0,1,0,0,0,2,0,1,1,1,0,1,1,1,0,1,2,0,1,1,0,0,2,2,0,1,1,1,0,1,0,0,0,2,2,0,1,1,2,1,0,1,1,2,1,1,0,1


## Sorting Through the Users

In [34]:
def top_spending_users(items_sold: pd.DataFrame, prices: pd.DataFrame, quantities_sold: pd.DataFrame) -> list:
  '''1. map each user to the dollars they spent'''
  def calculate_spending():
    # 1. iterate through user_ids in SHEET 3
    user_ids = quantities_sold["user_id"]
    spendings_of_users = dict()
    #   2. Look up the items they bought [SHEET 1]
    for user_id in user_ids:
      item_ids = items_sold[items_sold["user_id"] == user_id]["id"].iloc[0]
      item_ids = str(item_ids).split(",")
      # 3. for each of those items:
      total_spent = 0
      for item_id in item_ids:
        # 1. look up the quantitiy that user the bought [3]
        quantities_items = (
            quantities_sold[quantities_sold["user_id"] == user_id]
        )
        quantity = float(quantities_items[int(item_id)].iloc[0])
        # 2. multiply quantity * unit. price [2] 
        unit_price = float(
            prices[prices["Item_id"] == int(item_id)]["price"]
        )
        total_spent += quantity * unit_price
      # 4. map the price in the dict, to a list of users, who have bought that much
      if not total_spent in spendings_of_users:
        spendings_of_users[total_spent] = [user_id]
      else:
        spendings_of_users[total_spent].append(user_id)
    return spendings_of_users 

  # calc how much the users spent
  spendings_of_users = calculate_spending()
  # 2. sort by the keys in the dict, greatest to leas
  top_spending = list()
  for spending_amt in sorted(list(spendings_of_users.keys())):
    users = spendings_of_users[spending_amt]
    # 3. merge all the users into another, larger list
    top_spending.extend(users)
  # 4. return the sorted list
  return top_spending

### Time Analysis

```
let u = # of users
let p = # of unique products

Complexity of each high-level step:
1. u * p
2. u*log(u) 
3. u
4. constant

Overall Time Complexity: O(u(p + log(u)))
```

### Space Analysis: 

```
O(u) --> both the user_ids, spendings_of_users, and top_spending variable grow asympotically w/ # of users

O(p) - at most, a user might have bought at least 1 of each distinct product (stored in item_ids)

Overall Space Complexity: O(u + p)
```

### Test the Solution

In [35]:
top_spending_users(items_sold, prices, quantities_sold)

[1000931,
 100873,
 1015557,
 1016370,
 1022728,
 1023667,
 1027720,
 1035892,
 1036177,
 1041764,
 1045033,
 104672,
 1056238,
 1064492,
 1067244,
 1070053,
 1076873,
 1080923,
 1093281,
 1095882,
 1099222,
 111632,
 1119150,
 1130095,
 1130679,
 1143765,
 1152464,
 1158246,
 1161000,
 1182035,
 1198176,
 120280,
 1215020,
 1215832,
 1218432,
 1221360,
 1222022,
 1224892,
 122813,
 1229425,
 1231885,
 1232338,
 1235296,
 125030,
 1253476,
 125512,
 1266055,
 1268358,
 1270245,
 1275620,
 129491,
 1297730,
 1297736,
 1300574,
 134092,
 134781,
 1357143,
 1362722,
 1363232,
 1365664,
 1372670,
 1373524,
 1374033,
 1379766,
 1381082,
 1390441,
 1392762,
 1393504,
 1393899,
 1398043,
 140425,
 1410067,
 141208,
 1417887,
 1422895,
 1426606,
 1427389,
 1429126,
 1441377,
 1449794,
 1459449,
 1478275,
 1479674,
 1482677,
 1488384,
 1492655,
 1493638,
 1497187,
 1499428,
 155231,
 163889,
 164101,
 168337,
 170979,
 176578,
 181934,
 187754,
 18819,
 200535,
 20181,
 209059,
 211797,
 219847